# Working with Replicate in Pixeltable

Pixeltable's Replicate integration enables you to access Replicate's models via the Replicate API.

### Prerequisites

- A Replicate account with an API token.

### Important notes

- Replicate usage may incur costs based on your Replicate plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.

First you'll need to install required libraries and enter a Replicate API token.

In [ ]:
%pip install -qU pixeltable replicate

In [ ]:
import os
import getpass

if 'REPLICATE_API_TOKEN' not in os.environ:
    os.environ['REPLICATE_API_TOKEN'] = getpass.getpass(
        'Replicate API Token:'
    )

Now let's create a Pixeltable directory to hold the tables for our demo.

In [ ]:
import pixeltable as pxt

# Remove the `replicate_demo` directory and its contents, if it exists
pxt.drop_dir('replicate_demo', force=True)
pxt.create_dir('replicate_demo')

## Chat completions

Create a Table: In Pixeltable, create a table with columns to represent your input data and the columns where you want to store the results from Replicate.

In [ ]:
from pixeltable.functions.replicate import run

# Create a table in Pixeltable and pick a model hosted on Replicate with some parameters

t = pxt.create_table('replicate_demo.chat', {'prompt': pxt.String})

input = {
    'system_prompt': 'You are a helpful assistant.',
    'prompt': t.prompt,
    # These parameters are optional and can be used to tune model behavior:
    'max_tokens': 300,
    'top_p': 0.9,
    'temperature': 0.8,
}
t.add_computed_column(
    output=run(input, ref='meta/meta-llama-3-8b-instruct')
)

In [ ]:
# Parse the response into a new column
t.add_computed_column(response=pxt.functions.string.join('', t.output))

In [ ]:
# Start a conversation
t.insert([{'prompt': 'What foods are rich in selenium?'}])
t.select(t.prompt, t.response).show()

## Image generation

Here's an example that shows how to use Replicate's image generation models with Pixeltable. We'll use the FLUX Schnell model.

In [ ]:
t = pxt.create_table('replicate_demo.images', {'prompt': pxt.String})

input = {'prompt': t.prompt, 'go_fast': True, 'megapixels': '1'}
t.add_computed_column(
    output=run(input, ref='black-forest-labs/flux-schnell')
)

In [ ]:
t.insert(
    [
        {
            'prompt': 'Draw a pencil sketch of a friendly dinosaur playing tennis in a cornfield.'
        }
    ]
)

In [ ]:
t.select(t.prompt, t.output).collect()

We see that Replicate returns our image as an array containing a single URL. To turn it into an actual image, we cast the string to type `pxt.Image` in a new computed column:

In [ ]:
t.add_computed_column(image=t.output[0].astype(pxt.Image))
t.select(t.image).collect()

### Learn more

To learn more about advanced techniques like RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://docs.pixeltable.com/howto/use-cases/rag-operations) tutorial.

If you have any questions, don't hesitate to reach out.